In [9]:
# This will show the best guess for the encoding
# import chardet

# with open(r"C:\Users\E1460340\Desktop\New\Dataset\Demantra\Demantra.txt", "rb") as f:
#     result = chardet.detect(f.read())
#     print(result) 

import pandas as pd
from pathlib import Path

folder_path = Path(r"C:\Users\E1460340\Desktop\New")

for file in folder_path.iterdir():
    if file.name.__contains__("Dataset"):
        for dataset in file.iterdir():
            # Dataframe for Actual
            if dataset.name.__contains__("Actual"):
                for actual_file in dataset.iterdir():
                    if actual_file.name.__contains__("Firm and Forecast Orders"):
                        data_actual_orders = pd.read_csv(actual_file, low_memory= False)
                    elif actual_file.name.__contains__("Shipment"):
                        data_actual_shipments = pd.read_csv(actual_file, low_memory= False)
                    else:
                        pass
            # Dataframe for Prior
            elif dataset.name.__contains__("Prior"):
                for actual_file in dataset.iterdir():
                    if actual_file.name.__contains__("Firm and Forecast Orders"):
                        data_prior_orders = pd.read_csv(actual_file, low_memory= False)
                    elif actual_file.name.__contains__("Shipment"):
                        data_prior_shipments = pd.read_csv(actual_file, low_memory= False)
                    else:
                        pass
            # Dataframe for Demand Plan
            elif dataset.name.__contains__("Demantra"):
                for file in dataset.iterdir():
                    if "Demantra" in file.name:
                        data_demand_plan = pd.read_csv(file, delimiter= "\t", encoding= "UTF-16", low_memory= False)
                    else:
                        pass
            # Dataframe for Adjustments
            elif dataset.name.__contains__("factAdjustments"):
                adjustments = pd.read_excel(dataset, sheet_name= "Adjustments")
            else:
                pass
    elif file.name.__contains__("Lookup"):
        for lookup in file.iterdir():
            lookup_ResOEM = pd.read_excel(lookup, sheet_name= "LOOKUP_RES_OEM")
            lookup_ResAppCode = pd.read_excel(lookup, sheet_name= "LOOKUP_RES_APPCODE")
            lookup_ResPlatform = pd.read_excel(lookup, sheet_name= "LOOKUP_RES_PLATFORM")
    else:
        pass

# For making Res OEM Report merging actual and prior data
data_actual_orders.rename(columns={'SCHEDULE_SHIP_DATE':'SHIP_DATE','ORDERED_QUANTITY':'SHIPPED_QUANTITY'}, inplace= True)
data_prior_orders.rename(columns={'SCHEDULE_SHIP_DATE':'SHIP_DATE','ORDERED_QUANTITY':'SHIPPED_QUANTITY'}, inplace= True)

Res_actual = pd.concat([data_actual_shipments,data_actual_orders],ignore_index= True)
Res_prior = pd.concat([data_prior_shipments,data_prior_orders], ignore_index= True)

Res_actual.rename(columns={'SHIPPED_QUANTITY':'ACTUAL'}, inplace= True)
Res_prior.rename(columns={'SHIPPED_QUANTITY':'PRIOR'}, inplace= True)

Res_data = pd.concat([Res_actual,Res_prior], ignore_index= True)

# Filtering data
Res_data = Res_data[(Res_data['CUSTOMER_TYPE'] == 'External') & (Res_data['PORBU'] == 'AC_PORBU')]
Res_data = Res_data.merge(lookup_ResPlatform, how= 'inner', on= 'PLATFORM')

# Adding column for SEGMENT
Res_data = Res_data.merge(lookup_ResAppCode, how= 'left', left_on= 'APPLICATION_CODE', right_on= 'APP_CODE').rename(columns={'SEGMENT_y':'SEGMENT'})
Res_data['SEGMENT'] = Res_data['SEGMENT'].fillna('OTHERS')

# Removing other columns
Res_df = Res_data[['SHIP_DATE', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER', 'PLATFORM', 'ACTUAL', 'PRIOR']].copy()

# Making columns for SHIP_MONTH and SHIP_WEEK (start of week ;'monday')
Res_df['SHIP_DATE'] = pd.to_datetime(Res_df['SHIP_DATE'], format= 'mixed').dt.strftime("%m/%d/%Y")
Res_df['SHIP_MONTH'] = pd.to_datetime(Res_df['SHIP_DATE']).dt.strftime('%m/1/%Y')
Res_df['SHIP_WEEK'] = pd.to_datetime(Res_df['SHIP_DATE']).dt.to_period('W').dt.start_time.dt.strftime("%m/%d/%Y")

# grouping the data for ['SHIP_MONTH', 'SHIP_WEEK', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER'] to reduce the row count.
Res_df = Res_df.groupby(['SHIP_MONTH', 'SHIP_WEEK', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER']).agg(ACTUAL= ('ACTUAL','sum'), PRIOR= ('PRIOR','sum')).reset_index()


# Working with the adjustments
    # Adding percentage of weekly value to monthly
total_monthly = Res_df.groupby(['SHIP_MONTH', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER']).agg(monthly_ACTUAL= ('ACTUAL','sum'), monthly_PRIOR= ('PRIOR','sum')).reset_index()
Res_df = Res_df.merge(total_monthly, how= 'left' ,on= ['SHIP_MONTH', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER'])
Res_df['%_ACTUAL'] = Res_df['ACTUAL'].div(Res_df['monthly_ACTUAL']).fillna(0)
Res_df['%_PRIOR'] = Res_df['PRIOR'].div(Res_df['monthly_PRIOR']).fillna(0)

    # Cleaning the adjustments to align for Res data
adjustments['SEGMENT'] = 'RESIDENTIAL'
adjustments['SHIP_MONTH'] = pd.to_datetime(adjustments['SHIP_DATE'], format= 'mixed').dt.strftime('%m/%d/%Y')
adjustments[['ACTUAL','PRIOR']] = adjustments[['ACTUAL','PRIOR']].fillna(0)
Res_adj = adjustments[['SHIP_MONTH' ,'SEGMENT' ,'LEGACY_ACCOUNT_NUMBER' ,'ACTUAL', 'PRIOR']]
Res_adj = Res_adj.groupby(['SHIP_MONTH' ,'SEGMENT' ,'LEGACY_ACCOUNT_NUMBER']).agg(adjustments_ACTUAL= ('ACTUAL','sum'), adjustments_PRIOR= ('PRIOR', 'sum')).reset_index()

    # Merging the df and adj to get the adjusted numbers for ACTUAL and PRIOR
Res_adj['SHIP_MONTH'] = pd.to_datetime(Res_adj['SHIP_MONTH'])
Res_df['SHIP_MONTH'] = pd.to_datetime(Res_df['SHIP_MONTH'])
df_adj = Res_df.merge(Res_adj, how= 'left', on= ['SHIP_MONTH' ,'SEGMENT' ,'LEGACY_ACCOUNT_NUMBER'])

df_adj[['adjustments_ACTUAL','adjustments_PRIOR']] = df_adj[['adjustments_ACTUAL','adjustments_PRIOR']].fillna(0)
df_adj['ACTUAL'] = (df_adj['%_ACTUAL'] * df_adj['adjustments_ACTUAL']) + df_adj['ACTUAL']
df_adj['PRIOR'] = (df_adj['%_PRIOR'] * df_adj['adjustments_PRIOR']) + df_adj['PRIOR']

    # Selecting only the important columns
data_orders_shipments = df_adj[['SHIP_MONTH', 'SHIP_WEEK', 'SEGMENT' , 'LEGACY_ACCOUNT_NUMBER', 'ACTUAL', 'PRIOR']]

# Working with demand plan file from demantra
# Cleaning the demand_plan dataframe
    # Column Renaming
data_demand_plan.rename(columns={'Site Cust':'LEGACY_ACCOUNT_NUMBER',
                                 'Application Code':'APPLICATION_CODE',
                                 'Time':'SHIP_MONTH',
                                 'Fiscal Week / Series':'SHIP_WEEK',
                                 'DP Plan Proj':'DP_PLAN'
                                 }, inplace= True)

    # Filter
data_dp = data_demand_plan[
    (data_demand_plan['SHIP_WEEK'] != 'Summary') & 
    (data_demand_plan['SHIP_MONTH'] != 'Summary') & 
    (data_demand_plan['DP_PLAN'] != '0')].copy()

    # Changing DP_PLAN value from string to number and removing the week number in SHIP_WEEK.
data_dp['DP_PLAN'] = data_dp['DP_PLAN'].apply(lambda x: x.replace(',',''))
data_dp['DP_PLAN'] = pd.to_numeric(data_dp['DP_PLAN'])
data_dp['SHIP_WEEK'] = data_dp['SHIP_WEEK'].apply(lambda x: x.split(' ')[1])

    # Merging the lookupAPPCODE to get the SEGMENT
data_dp = data_dp.merge(lookup_ResAppCode, how='left', left_on= 'APPLICATION_CODE', right_on= 'APP_CODE')

    # Making the final demand_plan dataframe to be append in data_orders_shipments.
data_dp = data_dp.groupby(['SHIP_MONTH', 'SHIP_WEEK', 'SEGMENT' , 'LEGACY_ACCOUNT_NUMBER']).agg(DP_PLAN = ('DP_PLAN','sum')).reset_index()

# Combining data_orders_shipments and data_dp
data_dp = data_dp.copy()
data_orders_shipments = data_orders_shipments.copy()
data_dp['SHIP_MONTH'] = pd.to_datetime(data_dp['SHIP_MONTH'])
data_dp['SHIP_WEEK'] = pd.to_datetime(data_dp['SHIP_WEEK'], format='%d-%b-%y')
data_orders_shipments['SHIP_MONTH'] = pd.to_datetime(data_orders_shipments['SHIP_MONTH'])
data_orders_shipments['SHIP_WEEK'] = pd.to_datetime(data_orders_shipments['SHIP_WEEK'])
data_complete = pd.concat([data_orders_shipments,data_dp], ignore_index= True)
data_complete = data_complete.groupby(['SHIP_MONTH', 'SHIP_WEEK', 'SEGMENT' , 'LEGACY_ACCOUNT_NUMBER']).agg(ACTUAL= ('ACTUAL','sum'), PRIOR= ('PRIOR','sum'), DP_PLAN= ('DP_PLAN','sum')).reset_index()

# exporting the clean data
data_complete.to_csv(folder_path / 'clean_data.csv', index= False)